In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import metrics
import myfm
from myfm import RelationBlock
import pandas as pd
from scipy import sparse as sps
from mapper import DefaultMapper

# read movielens 100k data.
from movielens100k_data import MovieLens100kDataManager

In [2]:
data_manager = MovieLens100kDataManager()
df_train, df_test = data_manager.load_rating(fold=1)

user_info = data_manager.load_userinfo().set_index('user_id')
user_info['age'] = user_info.age // 5 * 5
user_info['zipcode'] = user_info.zipcode.str[0]
user_info_ohe = OneHotEncoder(handle_unknown='ignore').fit(user_info)

movie_info, movie_genres = data_manager.load_movieinfo()
movie_info['release_year'] = [
    str(x) for x in movie_info['release_date'].dt.year.fillna('NaN')
] # hack to avoid NaN
movie_info = movie_info[['movie_id', 'release_year'] + movie_genres].set_index('movie_id')
movie_info_ohe = OneHotEncoder(handle_unknown='ignore').fit(movie_info.drop(columns=movie_genres))

date_be = OneHotEncoder(handle_unknown='ignore').fit(
    df_train.timestamp.dt.date.values.reshape(-1, 1)
)
def categorize_date(df):
    return date_be.transform(df.timestamp.dt.date.values[:, np.newaxis])

In [3]:
user_to_internal = DefaultMapper(df_train.user_id.values)
movie_to_internal = DefaultMapper(df_train.movie_id)

In [4]:
# Implement side information and flavor of SVD++
# We add "all users who have evaluated a movie in the train set" or
# "all movies rated by a user" as a feture of user/movie.
use_date = True # use date info or not
use_iu = True # use implicit user feature
use_ii = True # use implicit item feature
use_user_info = True # use user information
use_movie_info = True # use movie information

movie_vs_watched = dict()
user_vs_watched = dict()
for row in df_train.itertuples():
    user_id = row.user_id
    movie_id = row.movie_id
    movie_vs_watched.setdefault(movie_id, list()).append(user_id)
    user_vs_watched.setdefault(user_id, list()).append(movie_id)

if use_date:
    X_date_train = categorize_date(df_train)
    X_date_test  = categorize_date(df_test)
else:
    X_date_train, X_date_test = (None, None)

In [5]:
# setup grouping
feature_group_sizes = []
if use_date:
    feature_group_sizes.append(
        len(date_be.categories_[0]), # date
    )

feature_group_sizes.append(len(user_to_internal)) # user ids

if use_iu:
    feature_group_sizes.append(len(movie_to_internal))

if use_user_info:
    feature_group_sizes.extend([
        len(c) for c in user_info_ohe.categories_ # user attributes
    ])

feature_group_sizes.append(len(movie_to_internal)) # movie ids
                           
if use_ii:
    feature_group_sizes.append(
        len(user_to_internal) # all users who watched the movies
    )

if use_movie_info:
    feature_group_sizes.extend([
        len(c) for c in movie_info_ohe.categories_ # user attributes
    ])
    feature_group_sizes.append(len(movie_genres))

grouping = [ i for i, size in enumerate(feature_group_sizes) for _ in range(size)]

In [6]:
# given user/movie ids, add additional infos and return it as sparse
def augment_user_id(user_ids):
    Xs = []
    X_uid = sps.lil_matrix((len(user_ids), len(user_to_internal)))
    for index, user_id in enumerate(user_ids):
        X_uid[index, user_to_internal[user_id]] = 1
    Xs.append(X_uid)
    if use_iu:
        X_iu = sps.lil_matrix((len(user_ids), len(movie_to_internal)))
        for index, user_id in enumerate(user_ids):
            watched_movies = user_vs_watched.get(user_id, [])
            normalizer = 1 / max(len(watched_movies), 1) ** 0.5
            for uid in watched_movies:
                X_iu[index, movie_to_internal[uid]] = normalizer
        Xs.append(X_iu)
    if use_user_info:
        Xs.append(user_info_ohe.transform(user_info.reindex(user_ids)))
    return sps.hstack(Xs, format='csr')

def augment_movie_id(movie_ids):
    Xs = []
    X_movie = sps.lil_matrix((len(movie_ids), len(movie_to_internal)))
    for index, movie_id in enumerate(movie_ids):
        X_movie[index, movie_to_internal[movie_id]] = 1
    Xs.append(X_movie)
    
    if use_ii:
        X_ii = sps.lil_matrix((len(movie_ids), len(user_to_internal)))
        for index, movie_id in enumerate(movie_ids):
            watched_users = movie_vs_watched.get(movie_id, [])
            normalizer = 1 / max(len(watched_users), 1) ** 0.5
            for uid in watched_users:
                X_ii[index, user_to_internal[uid]] = normalizer
        Xs.append(X_ii)    
    
    if use_movie_info:
        Xs.append(movie_info_ohe.transform(movie_info.drop(columns=movie_genres).reindex(movie_ids)))
        Xs.append(sps.csr_matrix(movie_info.reindex(movie_ids)[movie_genres].values))
    return sps.hstack(Xs, format='csr')

## User Relation Block to express data
See [\[Rendle 2013\]](http://www.vldb.org/pvldb/vol6/p337-rendle.pdf) how comlexity dcrease drastically in this case (and most cases with bipartite graph structure).

In [7]:
# Create RelationBlock.
# https://docs.scipy.org/doc/numpy/reference/generated/numpy.unique.html
train_blocks = []
test_blocks = []
for source, target in [(df_train, train_blocks), (df_test, test_blocks)]:
    unique_users, user_map = np.unique(source.user_id, return_inverse=True)
    target.append(
        RelationBlock(user_map, augment_user_id(unique_users))
    )
    unique_movies, movie_map = np.unique(source.movie_id, return_inverse=True)
    target.append(
        RelationBlock(movie_map, augment_movie_id(unique_movies))
    )

In [8]:
fm = myfm.MyFMRegressor(rank=8)
fm.fit(
    X_date_train, df_train.rating.values, X_rel=train_blocks,
    grouping=grouping, X_test=X_date_test, X_rel_test=test_blocks,
        y_test=df_test.rating.values,
        n_kept_samples=195, n_iter=200);

alpha = 1.57 w0 = 3.81  rmse_this: 0.95 mae_this: 0.74: 100%|██████████| 200/200 [00:11<00:00, 17.32it/s]


In [9]:
test_predictions = fm.predict(X_date_test, test_blocks)

rmse = (
    (test_predictions - df_test.rating.values)**2
).mean() ** 0.5
mae = np.abs(test_predictions - df_test.rating).mean()

# Note the improvement from "id_only" case.
# Compare this with methods like ones in https://paperswithcode.com/sota/collaborative-filtering-on-movielens-100k
print('rmse={}, mae={}'.format(rmse, mae))

rmse=0.8862637601998858, mae=0.6963833823580675


In [10]:
# If we use the original data format, it takes much more!
X_original_format = []
if use_date:
    X_original_format.append(X_date_train)

X_original_format.extend(
    [rel.data[rel.original_to_block] for rel in train_blocks]
)

X_original_format = sps.hstack(X_original_format, format='csr')

fm_rawformat = myfm.MyFMRegressor(rank=8).fit(X_original_format, df_train.rating,
                                          grouping=grouping, n_iter=6, n_kept_samples=1)

alpha = 1.30 w0 = 3.71 : 100%|██████████| 6/6 [00:14<00:00,  2.48s/it]


In [11]:
# They shoud be same up to floating point artifact.
fm_rawformat.predictor_.samples[-1].V - fm.predictor_.samples[0].V

array([[-1.94289029e-16,  2.35922393e-16, -1.05471187e-15, ...,
        -1.11022302e-16, -2.12330153e-15, -1.68615122e-15],
       [-1.38777878e-17,  1.80411242e-15, -9.02056208e-16, ...,
         2.91433544e-16, -1.60982339e-15, -3.08086889e-15],
       [ 4.16333634e-15,  1.22124533e-15, -1.55431223e-15, ...,
         2.13717932e-15,  1.66533454e-15,  1.08246745e-15],
       ...,
       [ 1.49880108e-15,  8.32667268e-17,  1.04083409e-15, ...,
         2.01227923e-16,  7.02563008e-16,  9.15933995e-16],
       [ 8.20524204e-16,  4.16333634e-16,  1.13797860e-15, ...,
         2.49800181e-16,  1.27675648e-15,  1.08593690e-15],
       [ 2.85882429e-15, -2.60902411e-15, -2.94209102e-15, ...,
        -1.26981758e-15,  1.80411242e-16,  1.87350135e-15]])